# CH02: Unlimited Register Machine

**Student**: Jingyu Yan

I implemented a URM (Unlimited Register Machine) simulator in Python to verify if my instructions are correct, but it is simple and might have some bugs.

## Load URM Class

Load the URM class from the urm_simulation.py file.

In [ ]:
# %load urm_simulation.py
"""
Created by Jingyu Yan on 2023/11/25.
"""

import copy


class URM(object):
    """
    Implementation scheme for simulating an Unlimited Register Machine,
    realizing the computational logic of four types of instructions: zero, successor, copy, and jump.
    """

    @staticmethod
    def zero(registers, n):
        """
        Set the value of register number n to 0.
        """
        registers[n] = 0
        return registers

    @staticmethod
    def successor(registers, n):
        """
        Increment the value of register number n.
        """
        registers[n] += 1
        return registers

    @staticmethod
    def copy(registers, j, k):
        """
        Copy the value of register number j to register number k.
        """
        registers[k] = registers[j]
        return registers

    @staticmethod
    def jump(registers, m, n, q, current_line):
        """
        Jump to line 'q' if values in registers 'm' and 'n' are equal, else go to the next line.
        """
        if registers[m] == registers[n]:
            return q - 1  # Adjust for zero-based indexing
        else:
            return current_line + 1

    def __init__(self, instructions, initial_registers, ):
        """
        Initialize a URM (Unlimited Register Machine) model, requiring input of an instruction set and registers.
        :param instructions: Instruction set, constructed and passed in from outside.
        :param initial_registers: Registers, constructed and passed in from outside.
        """
        self.instructions = instructions
        self.initial_registers = initial_registers
        self.instructions.append(('END',))  # Include an exit instruction by default.
        for v in self.initial_registers:
            assert v >= 0, "Input must be a natural number"

    def forward(self, safety_count=1000):
        """
        Execute simulated URM (Unlimited Register Machine) operations;
        this is only a simulation and cannot achieve 'infinity'
        :param safety_count: Set a safe maximum number of computations to prevent the program from falling into an infinite loop (to protect my device).
        :return: Return all computation steps and description of instructions.
        """
        registers = self.initial_registers

        current_line = 0
        count = 0
        while current_line < len(self.instructions):
            assert count < safety_count, "The number of cycles exceeded the safe number."

            # Adjust current_line to align with the external indexing starting from 1.
            instruction = self.instructions[current_line]
            op = instruction[0]

            # Construct a description of the current instruction.
            instruction_str = f"{current_line + 1}: {op}" + "(" + ", ".join(map(str, instruction[1:])) + ")"

            if op == 'Z':
                registers = self.zero(registers, instruction[1])
                current_line += 1
            elif op == 'S':
                registers = self.successor(registers, instruction[1])
                current_line += 1
            elif op == 'C':
                registers = self.copy(registers, instruction[1], instruction[2])
                current_line += 1
            elif op == 'J':
                jump_result = self.jump(registers, instruction[1], instruction[2], instruction[3], current_line)
                if jump_result == -1:  # If the result of the jump is -1, then terminate the program.
                    break
                else:
                    current_line = jump_result
            elif op == 'END':
                break  # Terminate the program.
            count += 1

            # Use a generator to return the state of the registers and the current instruction.
            yield copy.deepcopy(registers), instruction_str

    def __call__(self, *args, **kwargs):
        """
        Forward
        """
        return self.forward(*args, **kwargs)


def urm_op(func):
    """
    Decorator to convert the function to op.
    """
    def wrapper(*args):
        function_name = func.__name__
        return (function_name, *args)

    return wrapper


@urm_op
def C():
    pass


@urm_op
def J():
    pass


@urm_op
def Z():
    pass


@urm_op
def S():
    pass


_END = "END"  # op: end (private)


### Example: sum(x, y) = x + y

Use URM instructions to verify whether the sum(x, y) = x + y function is correct.

In [6]:
def add_x_y_show(x, y):
    instructions = [
        C(2, 0),       # copy R2 to R0
        Z(2),          # set R2 to zero
        J(1, 2, 0),    # if R1 == R2 then break
        S(0),          # R0++
        S(2),          # R2++
        J(3, 3, 3),    # into loop
    ]
    # request 4 registers and initialize each one to zero
    num_of_registers = 4
    R = [0 for _ in range(num_of_registers)]
    R[1] = x  # set R1 = x
    R[2] = y  # set R2 = y
    print(R)
    # Create a URM model
    urm = URM(instructions=instructions, initial_registers=R)
    # exec, set safety_count = 100 
    # Since this is only a simulation, it is necessary to set this safety value 
    # to prevent infinite loops during the debugging process
    result_registers = urm(safety_count=100)
    steps = list(result_registers)
    # Print each step of the calculation process
    for step, command in steps:
        print(f"{step} - {command}")

    print("Result of sum({}, {}) = {}".format(x, y, steps[-1][0][0]))

Run sum(3, 2):

In [7]:
x = 3
y = 2
add_x_y_show(x, y)

[0, 3, 2, 0]
[2, 3, 2, 0] - 1: C(2, 0)
[2, 3, 0, 0] - 2: Z(2)
[2, 3, 0, 0] - 3: J(1, 2, 0)
[3, 3, 0, 0] - 4: S(0)
[3, 3, 1, 0] - 5: S(2)
[3, 3, 1, 0] - 6: J(3, 3, 3)
[3, 3, 1, 0] - 3: J(1, 2, 0)
[4, 3, 1, 0] - 4: S(0)
[4, 3, 2, 0] - 5: S(2)
[4, 3, 2, 0] - 6: J(3, 3, 3)
[4, 3, 2, 0] - 3: J(1, 2, 0)
[5, 3, 2, 0] - 4: S(0)
[5, 3, 3, 0] - 5: S(2)
[5, 3, 3, 0] - 6: J(3, 3, 3)
Result of sum(3, 2) = 5


### Example: double(x) = 2x

Verify again: whether the function double(x) = 2x is correct.

In [12]:
def double_show(x):
    instructions = [
        C(1, 2),        # copy R1 to R3
        J(2, 0, 0),     # if R3 == R0 then jump to 6
        S(0),           # R0++
        S(0),           # R0++
        S(2),           # R3++
        J(2, 2, 2),     # into loop
    ]
    registers_num = 4
    initial_registers = [0 for _ in range(registers_num)]
    initial_registers[1] = x  # put the x in R1
    print(initial_registers)
    urm = URM(instructions, initial_registers)
    # run 
    result_registers = urm(safety_count=100)
    steps = list(result_registers)
    for step, command in steps:
            print(step, command)
    print("Result of double({}) = {}".format(x, steps[-1][0][0]))
    
# Run double(6):
double_show(6)

[0, 6, 0, 0]
[0, 6, 6, 0] 1: C(1, 2)
[0, 6, 6, 0] 2: J(2, 0, 0)
[1, 6, 6, 0] 3: S(0)
[2, 6, 6, 0] 4: S(0)
[2, 6, 7, 0] 5: S(2)
[2, 6, 7, 0] 6: J(2, 2, 2)
[2, 6, 7, 0] 2: J(2, 0, 0)
[3, 6, 7, 0] 3: S(0)
[4, 6, 7, 0] 4: S(0)
[4, 6, 8, 0] 5: S(2)
[4, 6, 8, 0] 6: J(2, 2, 2)
[4, 6, 8, 0] 2: J(2, 0, 0)
[5, 6, 8, 0] 3: S(0)
[6, 6, 8, 0] 4: S(0)
[6, 6, 9, 0] 5: S(2)
[6, 6, 9, 0] 6: J(2, 2, 2)
[6, 6, 9, 0] 2: J(2, 0, 0)
[7, 6, 9, 0] 3: S(0)
[8, 6, 9, 0] 4: S(0)
[8, 6, 10, 0] 5: S(2)
[8, 6, 10, 0] 6: J(2, 2, 2)
[8, 6, 10, 0] 2: J(2, 0, 0)
[9, 6, 10, 0] 3: S(0)
[10, 6, 10, 0] 4: S(0)
[10, 6, 11, 0] 5: S(2)
[10, 6, 11, 0] 6: J(2, 2, 2)
[10, 6, 11, 0] 2: J(2, 0, 0)
[11, 6, 11, 0] 3: S(0)
[12, 6, 11, 0] 4: S(0)
[12, 6, 12, 0] 5: S(2)
[12, 6, 12, 0] 6: J(2, 2, 2)
Result of double(6) = 12


### Example: fibb(0) = 0, fibb(1) = 1, and fibb(a + 2) = fbb(a) + fibb(a + 1) for any natural a

Verify that the Fibonacci function is correct:

In [13]:
def fibb_urm(x):
    instructions = [
        J(1, 0, 0),  # 1. if R1 == 0 then fibb(0) = 0
        S(0),  # 2. set R0 = 2
        J(1, 0, 0),  # 3. if R1 = 1 then fibb(1) = 1
        S(2),  # 4. set R2 = 1

        J(1, 2, 0),  # 5. loop_1 if R1 == R2 then jump to the end

        S(2),  # 6. set k++
        C(0, 4),  # 7. set fibb(k-1) to R4
        Z(0),  # 8. set R0 to 0
        Z(5),  # 9. set R5 to 0

        C(4, 0),  # 10. copy R4 to 0
        J(5, 3, 15),  # 11. if R5 == R3 then jump to 15
        S(0),  # 12. set R0++
        S(5),  # 13. set R5++
        J(1, 1, 11),  # 14. do while
        C(4, 3),  # 15. copy fibb(k-1) for the current k to fibb(k-2) for the next k(k++)

        J(2, 2, 5),  # 16 do while
    ]
    
    num_of_registers = 6
    R = [0 for _ in range(num_of_registers)]
    R[1] = x  # set R1 = x
    # copy the initial register
    R_init = copy.deepcopy(R)
    urm = URM(instructions=instructions, initial_registers=R)
    # Using a URM simulator to calculate fibb(x) numbers involves a large amount of computation, 
    # thus a larger safety_count needs to be set to prevent premature termination.
    safety_count = 10000
    # run
    steps_command = urm(safety_count=safety_count)
    
    return R_init, steps_command

Display the URM simulation process of the Fibonacci sequence for x=5

In [16]:
x = 7
R_init, results = fibb_urm(x)
print(R_init)
value = -1
for step, command in results:
    value = step[0]
    print(step, command)
print("Result of ffib({}) = {}".format(x,value))

[0, 7, 0, 0, 0, 0]
[0, 7, 0, 0, 0, 0] 1: J(1, 0, 0)
[1, 7, 0, 0, 0, 0] 2: S(0)
[1, 7, 0, 0, 0, 0] 3: J(1, 0, 0)
[1, 7, 1, 0, 0, 0] 4: S(2)
[1, 7, 1, 0, 0, 0] 5: J(1, 2, 0)
[1, 7, 2, 0, 0, 0] 6: S(2)
[1, 7, 2, 0, 1, 0] 7: C(0, 4)
[0, 7, 2, 0, 1, 0] 8: Z(0)
[0, 7, 2, 0, 1, 0] 9: Z(5)
[1, 7, 2, 0, 1, 0] 10: C(4, 0)
[1, 7, 2, 0, 1, 0] 11: J(5, 3, 15)
[1, 7, 2, 1, 1, 0] 15: C(4, 3)
[1, 7, 2, 1, 1, 0] 16: J(2, 2, 5)
[1, 7, 2, 1, 1, 0] 5: J(1, 2, 0)
[1, 7, 3, 1, 1, 0] 6: S(2)
[1, 7, 3, 1, 1, 0] 7: C(0, 4)
[0, 7, 3, 1, 1, 0] 8: Z(0)
[0, 7, 3, 1, 1, 0] 9: Z(5)
[1, 7, 3, 1, 1, 0] 10: C(4, 0)
[1, 7, 3, 1, 1, 0] 11: J(5, 3, 15)
[2, 7, 3, 1, 1, 0] 12: S(0)
[2, 7, 3, 1, 1, 1] 13: S(5)
[2, 7, 3, 1, 1, 1] 14: J(1, 1, 11)
[2, 7, 3, 1, 1, 1] 11: J(5, 3, 15)
[2, 7, 3, 1, 1, 1] 15: C(4, 3)
[2, 7, 3, 1, 1, 1] 16: J(2, 2, 5)
[2, 7, 3, 1, 1, 1] 5: J(1, 2, 0)
[2, 7, 4, 1, 1, 1] 6: S(2)
[2, 7, 4, 1, 2, 1] 7: C(0, 4)
[0, 7, 4, 1, 2, 1] 8: Z(0)
[0, 7, 4, 1, 2, 0] 9: Z(5)
[2, 7, 4, 1, 2, 0] 10: C(4, 0)
[2, 7, 4, 

Verify the results of the URM simulation for fibb(x) from 0 to 10:

In [17]:
x_set = [n for n in range(11)]
for x in x_set:
    _, results = fibb_urm(x)
    value = 0
    for step, command in results:
        value = step[0]
    print("Result of ffib_urm({}) = {}".format(x,value))

Result of ffib_urm(0) = 0
Result of ffib_urm(1) = 1
Result of ffib_urm(2) = 1
Result of ffib_urm(3) = 2
Result of ffib_urm(4) = 3
Result of ffib_urm(5) = 5
Result of ffib_urm(6) = 8
Result of ffib_urm(7) = 13
Result of ffib_urm(8) = 21
Result of ffib_urm(9) = 34
Result of ffib_urm(10) = 55


Implement a Python function for verification.

In [18]:
def fibonacci_recursive(n):
    if n <= 1:
        return n
    else:
        return fibonacci_recursive(n - 1) + fibonacci_recursive(n - 2)

# Use x_set
for i in x_set:
    x = fibonacci_recursive(i)
    print(f'fibb({i}) = {x}')

fibb(0) = 0
fibb(1) = 1
fibb(2) = 1
fibb(3) = 2
fibb(4) = 3
fibb(5) = 5
fibb(6) = 8
fibb(7) = 13
fibb(8) = 21
fibb(9) = 34
fibb(10) = 55


### Example: pred(x) = x - 1

In [20]:
x = 5
instructions = [
    J(0, 1, 0),
    S(2,),
    J(1, 2, 0),
    S(0),
    S(2),
    J(0, 0, 3)
]
registers_num = 4
initial_registers = [0 for _ in range(registers_num)]
initial_registers[1] = x  # put the x in R1
print(initial_registers)
urm = URM(instructions, initial_registers)
# run 
result_registers = urm(safety_count=1000)
steps = list(result_registers)
for step, command in steps:
        print(step, command)
print("Result of pred({}) = {}".format(x, steps[-1][0][0]))

[0, 5, 0, 0]
[0, 5, 0, 0] 1: J(0, 1, 0)
[0, 5, 1, 0] 2: S(2)
[0, 5, 1, 0] 3: J(1, 2, 0)
[1, 5, 1, 0] 4: S(0)
[1, 5, 2, 0] 5: S(2)
[1, 5, 2, 0] 6: J(0, 0, 3)
[1, 5, 2, 0] 3: J(1, 2, 0)
[2, 5, 2, 0] 4: S(0)
[2, 5, 3, 0] 5: S(2)
[2, 5, 3, 0] 6: J(0, 0, 3)
[2, 5, 3, 0] 3: J(1, 2, 0)
[3, 5, 3, 0] 4: S(0)
[3, 5, 4, 0] 5: S(2)
[3, 5, 4, 0] 6: J(0, 0, 3)
[3, 5, 4, 0] 3: J(1, 2, 0)
[4, 5, 4, 0] 4: S(0)
[4, 5, 5, 0] 5: S(2)
[4, 5, 5, 0] 6: J(0, 0, 3)
Result of pred(5) = 4
